 <center><img src=img/MScAI_brand.png width=70%></center>

# Floating-point numbers



As we know, computer languages can't represent *real* numbers perfectly, because a real number can take on an infinite number of possible values (even in $[0, 1]$).

Let's say we are using the standard `float` type in Python, which uses 64 bits to represent a number.

<center><img src=img/machine_epsilon.svg width=50%></center>

 Consider just a tiny subset of the number line as shown. Let's say it starts at $x$ represented by some bitstring of length 64. There has to be some number which is the next largest, out of all possible bitstrings of length 64. Call that *successor(x)*. Well, in between there are still infinitely many (in fact, uncountably many) other numbers which can't be represented in our 64-bit system! Any of them can only be rounded off to either $x$ or *successor(x)*.

Floating-point, specified by IEEE 754, is the most common scheme for representing real numbers. Single-precision numbers (e.g. `float` in the C language) use 32 bits each, and double-precision (`double`) use 64. In Python all floating-point numbers are double-precision but are just called `float`.

### FP is inexact

Floating-point calculations can be inexact:

In [17]:
2 / 3

0.6666666666666666

By the way, if you want C-style integer division, it is available using `//`:

In [18]:
2 // 3

0

A consequence of this inexactness is that `x == y` is not meaningful for `float` values. 

In [19]:
import math
import numpy as np
x = math.sqrt(2) ** 2
y = 2.0
print(x, y)

2.0000000000000004 2.0


In [20]:
x == y

False

Instead we usually use a construction like this:

In [21]:
eps = 10**-8 # epsilon, a small constant which we take as "close enough"
abs(x - y) < eps

True

This is testing whether `x` and `y` are "close enough". 

Numpy has `np.allclose` for this:

In [22]:
x = np.linspace(0.0, 1.0, 10, endpoint=False) / 3.0

In [23]:
x

array([0.        , 0.03333333, 0.06666667, 0.1       , 0.13333333,
       0.16666667, 0.2       , 0.23333333, 0.26666667, 0.3       ])

In [24]:
y = np.linspace(0.0, 0.33333333, 10, endpoint=False)

In [25]:
y

array([0.        , 0.03333333, 0.06666667, 0.1       , 0.13333333,
       0.16666666, 0.2       , 0.23333333, 0.26666666, 0.3       ])

In [26]:
x - y

array([0.00000000e+00, 3.33333333e-10, 6.66666666e-10, 1.00000001e-09,
       1.33333333e-09, 1.66666667e-09, 2.00000003e-09, 2.33333336e-09,
       2.66666667e-09, 2.99999997e-09])

In [27]:
np.allclose(x, y)

True

### FP has upper and lower bounds on magnitude

Some calculations just become too large, and we can get an `OverflowError`:

In [28]:
print("ok:", (10.0**10.0)**10.0)
print("not ok:", 10.0**(10.0**10.0))

ok: 1e+100


OverflowError: (34, 'Result too large')

*Machine epsilon* is the academic term for *the smallest magnitude that can be represented* in a floating-point system - the gap between two successive `float`s. (More strictly: the gap between 0.0 and the next `float`.)

**Exercise**: what is machine epsilon in Python?

In [29]:
# HINT
print(10 ** -10)
print(10 ** -350)

1e-10
0.0


### Special numbers

IEEE 754 allows for three special numbers: infinity (`inf`), negative infinity (`-inf`), and *not a number* (`nan`). In principle these can arise through division by zero and similar operations, but in practice Python just throws an error instead:

In [30]:
z = 15 / 0 # would be "inf"

ZeroDivisionError: division by zero

In [31]:
0 / 0 # would be "nan"

ZeroDivisionError: division by zero

However, we can create these by hand and look at their properties:

In [32]:
inf = float('inf')
neg_inf = -inf
nan = float('nan')

In [33]:
inf > 3

True

In [34]:
-inf < inf

True

In [35]:
inf + -inf # don't expect zero, here!

nan

In [36]:
inf + 3

inf

In [37]:
nan < 3

False

### Floating-point errors in Numpy

All of the above is using pure Python. The story becomes slightly different in Numpy. Numpy uses the same floating-point system as pure Python. However, its method of handling errors is different, because in Numpy it is common to process an entire array at once, and an error (e.g. division by zero) might occur only for some elements of the array. So, Numpy allows you to *choose* how errors are handled.

In [38]:
import numpy as np
np.seterr(divide="warn") # see https://docs.scipy.org/doc/numpy/reference/generated/numpy.seterr.html

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

Here, `divide="warn"` says: if a divide-by-zero error happens, please warn me, but allow the result to be calculated, so I can see it.

In [39]:
x = 1.0
y = np.array([0.0, 1.0, 2.0]) # 1 / 0 -> inf
x / y

<ipython-input-39-d0b6071f7bf0>:3: RuntimeWarning: divide by zero encountered in true_divide
  x / y


array([inf, 1. , 0.5])

In [40]:
x = 0.0
y = np.array([0.0, 1.0, 2.0]) # 0 / 0 -> nan
x / y

<ipython-input-40-53ee1611e570>:3: RuntimeWarning: invalid value encountered in true_divide
  x / y


array([nan,  0.,  0.])